In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 1, 14), datetime.date(2023, 1, 13))

In [2]:
# specify the number of business days
num_days = 1
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(num_days)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-01-14
yesterday: 2023-01-13 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
yesterday, a_year_ago

(datetime.date(2023, 1, 13), datetime.date(2022, 1, 13))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [4]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,0


In [5]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

0

In [6]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'BANPU', 'BCH', 'CKP', 'CPF', 'CPNREIT', 'GVREIT', 'IVL', 'JMART', 'MAKRO', 'ORI', 'PTT', 'PTTGC', 'SCC', 'SINGER', 'SYNEX', 'TFFIF', 'TFG', 'TMT', 'WHAIR'"

In [7]:
in_p = "'ASP'"

In [8]:
len(in_p.split(','))

1

### Get past one year data

In [9]:
sql = """
SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (a_year_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '2022-01-13' AND name IN ('ASP') 
ORDER BY name, date


In [10]:
df = pd.read_sql(sql, const)
df.head()

,name,date,price,qty,maxp,minp
0,ASP,2022-01-13,3.80,88763239,3.92,3.76
1,ASP,2022-01-14,3.96,193037099,4.20,3.80
2,ASP,2022-01-17,3.86,35303941,4.02,3.84
3,ASP,2022-01-18,3.86,39975369,3.98,3.82
4,ASP,2022-01-19,3.88,15774275,3.88,3.78


In [11]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)